# **Project 0:** Random Garden, 
This notebook is a full example of the process of reading in and working with data, as well as how to apply k nearest neighbor and k means to data. There are examples included that help to showcase, plotting, DataFrames, json files, np.arrays as well as many other python tools. In addition to these python elements, we also have the integration and utilization of the a production version of the code from class. The result of the code allows us to compare KNN and Kmeans and reflect on the differences as we start to build foundational intuition around these two tools.   

---
Overview:
- Data read in (both data sets)
    - Classical Flower Set --> `/Data/biology/Iris.csv`
    - Random Garden Data --> `/Data/biology/Garden_Data.csv`
- Display data. 
- K-Means
    - Save result
- KNN
    - Save result
- Compare the results. 
- Reflect.
---

In [ ]:
# Allows us to use custom modules
import sys
sys.path.append("/home/Files/") 

import numpy as np # Math.
import matplotlib.pyplot as plt # Plotting.
import pandas as pd # Data.
import json #Dictionary style file that allows for key word search

# Custom module made from a rebuild of the lectures code (Christian and Joe).
from rrccml.KNN import KNN
from rrccml.KMeans import kmeans

## **Data Read** (Feb 8th/9th)

_Scramble 0.1:_ Read in both data sets as `Dataframes` and assign them to two variables. (Paths to the data above)

In [ ]:
# Read in the unlabeled garden data and the labeled iris data.
garden_raw_data = pd.read_csv("~/Data/biology/Garden_Data.csv", names=["Feature_1","Feature_2","Feature_3","Feature_4"])
iris_raw_data = pd.read_csv("~/Data/biology/Iris.csv")

_Scramble 0.2:_ Print some of the data from each of the sets. Take notes to share on what you see.

In [ ]:
# Print the start of the garden_raw_data dataframe.
garden_raw_data.head() # Pandas head() method is used to return top n (5 by default) rows of a data frame or series.

In [ ]:
# Rename the string species as 0 , 1 , 2.
iris_raw_data["Num_Cat"] = iris_raw_data["Species"].map({"Iris-setosa" : 0, "Iris-versicolor": 1,  'Iris-virginica': 2}) #map() function returns a map object(which is an iterator) of the results after applying the given function 
# to each item of a given iterable (list, tuple etc.)

In [ ]:
# Print the keys for the data.
iris_raw_data.keys() #keys() method in Python Dictionary, returns a view object that displays a list of all the keys in the dictionary.

In [ ]:
# Assign iris_raw_data to just the features and the numbers for the species.
iris_raw_data = iris_raw_data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Num_Cat']]
iris_raw_data

_Scramble 0.3:_ Name the features (columns) of the `Garden_Data.csv`.

In [ ]:
# This was done in the readin line above.

_Scramble 0.4:_ Save the data frame using. 
```Python
pandas.DataFrame.to_csv
```

In [ ]:
# This will save a csv file with the column-labeled garden data next to the notebook.
pd.DataFrame.to_csv(garden_raw_data, "Garden_Data_Labeled.csv")

## **Data Display**

_Scramble 0.5:_ Plot a scatter plot. Of two of the features using. Once you have one set plotted, plot other features with respect to each other. 

```Python
plt.scatter(f1,f2)
```


In [ ]:
plt.scatter(garden_raw_data["Feature_1"], garden_raw_data["Feature_2"])
plt.show()

In [ ]:
plt.scatter(iris_raw_data["PetalLengthCm"], iris_raw_data["PetalWidthCm"], c = iris_raw_data["Num_Cat"]) # What is this graphing?
plt.show()

---

## **KNN Integration** (Feb 12th)

This next cell take 5 inputs. The labeled data, the number of times we should loop over all the data, the number of features, the percent of data to be training, the k's to range over. 

```Python
KNN.determine_k(DataFrame, times_through_all_data, number_of_features, percentage, [low_k, high_k])
```

The output of this function is the scores for each of the `times_through_all_data` and the best `k` overall.

In [ ]:
# determine_k was written in a previous lecture K_Means (I think)?

### **Exercise 1:** 

Change the values of, `times_through_all_data`, `number_of_features`, `percentage`, `[low_k, high_k]`. Run this cell a few times and take a look at the output. 

Questions:
- Why does it take so long to run this cell?
- How many features can we have for the iris data set?
- Do we need to check all `k` values to 150? Looking at the images, what should the `high_k` be.
- What does it look like the best value for `k` is?


In [ ]:
general_overview, k = KNN.determine_k(pd.DataFrame(iris_raw_data), 20, 2, .60, [1,90]) #Runs through data 20 times looking for k's, 2 features examined
# The 150 is to run through all 150 data points, but as we see, it drops off at about 90, so could reduce 150 to 90 to reduce computation time.
# Look for graph that gives best accuracy or average the trials, so getting to about 90 nearest neighbors, the accuracy drops to 0
# Each test trials resamples testing and training data
# 20 runs through 20 times. We can change this.This helps us determine the best number of nearest neighbors
# 2 is number of geatures we're going to look at.
# .60 is the percent for training
# tqdm gives status bar, nice to know program hasn't stalled
# Every line of source code has been commented  if you check Contextual Help
# Change the numbers and see what happens
# ALL OF THIS IS DETERMINE THE BEST k!!!

The `general_overview` is the collections of all the scores for each k for all the `times_through_all_data`. The goal is to find the best `k` for the data, so considering all the runs together can help to find this.

While there are many ways to find this "best" `k`, the way that the code here finds this averaging the best performing `k` for each of the runs.

### **Exercise 2:**
- What do each of the values in the shape of `general_overview` represent?
- `print(k)` tells us the best `k`. What does this `k` mean?

In [ ]:
#print(general_overview)
#print(k) #This gives accuracy so 1 is 93%, 2 is 86%
# (print shape gives array numbers)# Numbers below mean 20 iterations, 0-149 data points, 

In [ ]:
print(np.shape(general_overview))
print(k) #This shows number of trials, the number of data points considered and the 2 columns of "neighbor" and "accuracy"
# So below, the 11 is the determined number of nearest neighbors, or k

In the next cell we will classify each of the data point of the `Garden_Data.csv'`. The `knn` method allows us to pass it a point that we need classified and a data set to compare against. This method also allows set the value of `k`. We can use the one that we found above. 

```Python
KNN.knn( point_to_classify , [classified_data_features, classified_data_targets] , k)
```

In [ ]:
# KNN.knn () This part is DEPLOYING!! THIS USES THE BEST k WE FOUND AND EMPLOYS IT!!
# points to classify: these are unclassified data points, takes a single
# Number of features to include. 
f = 4

# Make the iris_raw_data into a numpy array, which has name tags (classified)
iris_raw_data_array = np.array(iris_raw_data)

# List to collect the classes for each datapoint.
classes = []

# Loop over all the garden data, which does not have name tags (unclassified)
for x in np.array(garden_raw_data): #Takes all data from garden data and puts into method (compares to specified labeled neighbors )
    classes.append(int(KNN.knn( x, [iris_raw_data_array[:, :f], iris_raw_data_array[:, -1]] , k)))
    # best k value we discovered was "29" as seen above, could also put "k" so each time it was calculated, it would stick it in here
    # ex     classes.append(int(KNN.knn( x, [iris_raw_data_array[:, :f], iris_raw_data_array[:, -1]] , k)))

In [ ]:
# Cast the classes into an array.
Guess = np.array(classes)

# Show how many of each classification we have found.
print(np.unique(Guess, return_counts=True)) #unique command, will tell unique values number of elements represented
#Below this array says there are 200 of one type, 197 of another and 203 of a third. This is pretty accurate since there were 200 of each.

The next cell reads in the answer key and compares our findings with the key.

### **Exercise 3:**
What does the output of,

```Python
np.unique(np.array(Guess) - np.array(Answer) ,return_counts=True)
```

do?



In [ ]:
Answer = np.loadtxt("/home/stacie_barbarick/Data/biology/Garden_Data_Key.csv", dtype=float)
print(np.unique(np.array(Guess) - np.array(Answer) ,return_counts=True)) #Took answer key subtract guess, if answer is 0, it was correct

In [ ]:
585/600 #This shows that 576 of the 600 data points were correct, or 97.5%

## **K Means Integration**

In [ ]:
# Reminder: K Means unsuoervised learning, creates clusters, finds centroids and measures distances from data point to centroid,
# iteratively; used elbow function to find number of clusters, k.

For K means we are going to use a configuration .json to control the setting for the algorithm. Often times with machine learning, using a file to control all the setting can help 
the process of running the code since the user won't need to change the variables in the source code, or answer inputs. 

The configuration for the K means structure is:

```Python
config = {"title": " ", <--- # Title of the configuration
        "data_path": "", <--- # Path to the data.
        "col_names": [], <--- # Names for the columns
        "toggle_col_names": True, <--- # Do the columns have names already.
        "k_range": [1, 10], <--- # Range of values for k. 
        "f": [1,4], <--- # Location of the features in the data. (counting from 0)
        "each_k": 1} <--- # Number of times to check each k. 
```
This lets us set all the variables and functions before we start
Now with this file we have a few choices. We could make it in a text editor and aim the algorithm at it. Or we can generate here with python and have it save it locally. Both options are totally valid, but it the spirit of everything for this project being in this notebook, we are going to generate these configurations here. 

### **Exercise 4:**
In the below cell the first configuration is all set. The `toggle_col_names` is True, since the `Iris.csv` data set has labels already. For the exercise, add the missing information for the second configuration. 

Notes about the `Garden_Data.csv`:
- the original csv does not have labels.
- the features are in columns 0,1,2,3.
- the number of clusters is between 1 and 10. (This will be important for our k_range)



In [ ]:
config = {"title": "Iris Configuration", # The setup configuration file is a useful middle-ground between the setup script—which, ideally, would be opaque to installers
        "data_path": "~/Data/biology/Iris.csv",
        "col_names": [],
        "toggle_col_names": True,
        "k_range": [1, 10],
        "f": [1,4],
        "each_k": 1}

with open('./iris_kmeans_config.json', 'w') as outfile:
    json.dump(config, outfile)
    
config2 = {"title": "Random Garden Data Configuration",
        "data_path": "~/Data/biology/Garden_Data.csv",
        "col_names": ["Feature_1", "Feature_2", "Feature_3", "Feature_4"],
        "toggle_col_names": False,
        "k_range": [1, 10],
        "f": [0, 3],
        "each_k": 1}
# k_range and f need to have the same values.
with open('./RG_kmeans_config.json', 'w') as outfile:
    json.dump(config2, outfile) #Will have to toggle between json files, these files have been created in my main directory

This next cell will apply k-means to the iris original data, as well as the random garden data. The output of this method will include all the tools that we need to determine the `k` that we should use as well as a few way to save and interact with the clusters. 

In [ ]:
K_Iris = kmeans.kmeans(config="iris_kmeans_config.json")
K_Random_Garden = kmeans.kmeans(config="RG_kmeans_config.json")

### **Exercise 5:**
The methods that kmeans has are: 
    
    Methods
    -------
    save_clusters : np.array
        returns labeled points
    show_clusters : None
        plots clusters
    show_variance : None
        plots variance
    _run : None
        generates model
        
View the contextual help (right click in a python cell and choose it from the menu) Once this is open, click on the `kmeans` (the one on the right) in, 
```Python 
kmeans.kmeans(config="iris_kmeans_config.json") 
```
then click on the `Show Contextual Help` tab. You should see the method that we have above in this tab.

Each of these methods will allow us to access some of the information that we computed above when we ran,

```Python
K_Iris = kmeans.kmeans(config="iris_kmeans_config.json")
K_Random_Garden = kmeans.kmeans(config="RG_kmeans_config.json")
```

Let's take a look at some of this information.

### **Exercise 6**

Run the cells below and consider the elbow graph for each. 

Looking at how the variance for the different amounts of clusters can help us to determine the number of clusters that we should use. This is called [The Elbow Method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)). In the graphic we are looking for the cluster number where the variance of the data in the clusters settles.  

What cluster number should we use with the elbow method in mind?

In [ ]:
K_Iris.show_variance()


In [ ]:
K_Random_Garden.show_variance()


### **Exercise 7:**
The `save_clusters()` method allows us to view and save the clusters that we have computed. And while we may not have labels for the data (unsupervised) we can append the cluster label to each data value and see how the grouping look.

The method `save_clusters()` take a few arguments, the first of which is the cluster number that you would like to view. 

- Set the cluster number to a few different values. Make sure it is an integer in the range that we set in the configuration above. (Will be 1 to 10 if we used the same as the given configuration)
- Choose the cluster number that we determined with the above elbow graph and run the cell one more time.
- After running the `save_clusters()` with the `k` from the elbow analysis, run the cells that follow to see the clustered data. 

In [ ]:
Clusters_Iris = K_Iris.save_clusters(3)
Clusters_RG = K_Random_Garden.save_clusters(3)

In [ ]:
Clusters_Iris

In [ ]:
Clusters_RG

---
### **Compare Results** 

The final few cells have us read in all the missing data and look at some plots of all our finding from KNN and Kmeans. 

### **Exercise 8:**

Run all the cells that follow and take some time to consider the graphs. Be thinking about the difference between KNN and Kmeans and what applications one might have over the other. 

_Note: You may need to run some of the cell in the data read-in section as well as the KNN section. This brings everything together, so all the data and processing will be necessary._ 

In [ ]:
iris_data = pd.read_csv("~/Data/biology/Iris.csv")

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True)
fig.set_size_inches(18.5, 10.5)

ax1.scatter(Clusters_Iris[0], Clusters_Iris[1], c= Clusters_Iris["Cluster_Index"])
ax1.set_title("K means grouping on Iris data", fontsize=8)
#plt.title("K means grouping on Iris data", fontsize=8)

ax2.scatter(Clusters_RG[0], Clusters_RG[1], c= Clusters_RG["Cluster_Index"])
ax2.set_title("K means grouping on Random Garden data", fontsize=8)

ax3.scatter(iris_data["SepalLengthCm"], iris_data["SepalWidthCm"], c= pd.factorize(iris_data["Species"])[0])
ax3.set_title("Real Iris Data Grouping", fontsize=8)

ax4.scatter(garden_raw_data["Feature_1"], garden_raw_data["Feature_2"], c= Answer)
ax4.set_title("Real Random Garden Data Grouping", fontsize=8)

plt.show()


In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True)
fig.set_size_inches(18.5, 10.5)

ax1.scatter(garden_raw_data["Feature_1"], garden_raw_data["Feature_2"], c= Answer)
ax1.set_title("Real Random Garden Data Grouping", fontsize=8)

ax2.scatter(Clusters_RG[0], Clusters_RG[1], c= Clusters_RG["Cluster_Index"])
ax2.set_title("K means grouping on Random Garden data", fontsize=8)

ax3.scatter(garden_raw_data["Feature_1"], garden_raw_data["Feature_2"], c= Guess)
ax3.set_title("K Nearest Neighbor Random Garden Data Classification", fontsize=8)

ax4.scatter(iris_data["SepalLengthCm"], iris_data["SepalWidthCm"], c = pd.factorize(iris_data["Species"])[0])
ax4.set_title("Real Iris Data Grouping", fontsize=8)

ax5.scatter(Clusters_Iris[0], Clusters_Iris[1], c= Clusters_Iris["Cluster_Index"])
ax5.set_title("K means grouping on Iris data", fontsize=8)
#plt.title("K means grouping on Iris data", fontsize=8)

plt.show()

---
---